# Session 10: Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

## 🤝 Breakout Room #1
  - Task 1: Installing Required Libraries
  - Task 2: Set Environment Variables
  - Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  - Task 4: Construct our RAG application
  - Task 5: Evaluating our Application with Ragas
  - Task 6: Making Adjustments and Re-Evaluating
  - ***Activity #1: Implement a Different Reranking Strategy***


## Task 1: Installing Required Libraries

If you have not already done so, install the required libraries using the uv package manager:
``` bash

uv sync

```


## Task 2: Set Environment Variables:

We'll also need to provide our API keys.
> NOTE: In addition to OpenAI's models, this notebook will be using Cohere's Reranker - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

You have two options for supplying your API keys in this session:
- Use environment variables (see Prerequisite #2 in the README.md)
- Provide them via a prompt when the notebook runs

The following code will load all of the environment variables in your `.env`. Then, it checks for the two API keys we need. If they are not there, it will prompt you to provide them.

First, OpenAI's for our LLM/embedding model combination!

Second, Cohere's for our reranking


In [1]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

## Task 3: Synthetic Dataset Generation for Evaluation using Ragas

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data using the Health & Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, and stress management.

Next, let's load our data into a familiar LangChain format using the `TextLoader`.

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/HealthWellnessGuide.txt")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [3]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [4]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/4 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/9 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

In [5]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,how i do partial crunches for help my back pai...,[The Personal Wellness Guide A Comprehensive R...,Partial Crunches: Lie on your back with knees ...,single_hop_specifc_query_synthesizer
1,How can the Cat-Cow Stretch be incorporated in...,[The Personal Wellness Guide A Comprehensive R...,The Cat-Cow Stretch is recommended for lower b...,single_hop_specifc_query_synthesizer
2,How can chamomile be used to support better sl...,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,Chamomile can be consumed as an herbal tea as ...,single_hop_specifc_query_synthesizer
3,What key information does Chapter 7 provide ab...,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,Chapter 7 explains that sleep is crucial for p...,single_hop_specifc_query_synthesizer
4,wat tips does Chapter 17 give for digestive he...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 17 suggests supporting digestive healt...,single_hop_specifc_query_synthesizer
5,Wut are the main tipz and foodz mentiond in Ch...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 17: Digestive Health explains that a h...,single_hop_specifc_query_synthesizer
6,"According to Chapter 9 and Chapter 18, how can...",[<1-hop>\n\nPART 3: SLEEP AND RECOVERY Chapter...,Improving sleep can help boost immune function...,multi_hop_specific_query_synthesizer
7,What practical strategies from Chapter 9 can h...,[<1-hop>\n\nPART 3: SLEEP AND RECOVERY Chapter...,Chapter 9 suggests practical strategies for ma...,multi_hop_specific_query_synthesizer
8,How can understanding the science of sleep fro...,[<1-hop>\n\nPART 3: SLEEP AND RECOVERY Chapter...,"To develop and maintain a consistent, high-qua...",multi_hop_specific_query_synthesizer
9,How can the understanding of insomnia from Cha...,[<1-hop>\n\nPART 3: SLEEP AND RECOVERY Chapter...,Chapter 9 explains that insomnia involves diff...,multi_hop_specific_query_synthesizer


## Task 4: Construct our RAG application

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [8]:
loader = TextLoader("data/HealthWellnessGuide.txt")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

447

### ❓ Question #1:

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

##### Answer:

Chunk overlap indicates - how much text is shared between consecutive chunks during splitting.
The main purpose of chunk overlap is to minimize information loss at chunk boundaries, and increases the chance that the retrieved chunk still contains the full context.


Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [11]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [12]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [13]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

Now we can produce a node for retrieval!

In [14]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### A - Augmented

Let's create a simple RAG prompt!

In [15]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### G - Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-nano` to avoid using the same model as our judge model.

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

Then we can create a `generate` node!

In [17]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [18]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [19]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [20]:
response = graph.invoke({"question" : "What exercises help with lower back pain?"})

In [21]:
response["response"]

'The provided context does not specify any particular exercises that help with lower back pain.'

## Task 5: Evaluating our Application with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [22]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [23]:
dataset.samples[0].eval_sample.response

'The provided context does not include specific steps or instructions on how to do partial crunches or details about the number of repetitions. For proper guidance on performing partial crunches to help your back pain, it is best to consult a healthcare professional or a certified physical therapist. They can give you personalized advice and correct technique to ensure safety and effectiveness.'

Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [24]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [25]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

Next up - we simply evaluate on our desired metrics!

In [26]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

baseline_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
baseline_result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

{'context_recall': 0.2887, 'faithfulness': 0.5581, 'factual_correctness': 0.4082, 'answer_relevancy': 0.4438, 'context_entity_recall': 0.2645, 'noise_sensitivity_relevant': 0.0000}

## Task 6: Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!




We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data_new_chunks",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data_new_chunks",
    embedding=embeddings,
)

_ = vector_store.add_documents(documents=split_documents)

adjusted_example_retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [28]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=adjusted_example_retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [29]:
class AdjustedState(TypedDict):
  question: str
  context: List[Document]
  response: str

adjusted_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_adjusted, generate])
adjusted_graph_builder.add_edge(START, "retrieve_adjusted")
adjusted_graph = adjusted_graph_builder.compile()

In [30]:
response = adjusted_graph.invoke({"question" : "How can I improve my sleep quality?"})
response["response"]

'To improve your sleep quality, consider adopting good sleep hygiene practices such as maintaining a consistent sleep schedule, creating a relaxing bedtime routine, and keeping your bedroom cool, dark, and quiet. Limit screen exposure 1-2 hours before bed and avoid caffeine after 2 PM. Regular exercise is beneficial, but not too close to bedtime, and try to limit alcohol and heavy meals before sleep. Additionally, use tools like blackout curtains or a sleep mask, ensure your mattress and pillows are comfortable, and aim for 7-9 hours of sleep each night. Incorporating relaxation techniques, herbal teas like chamomile or valerian root, or practicing mindfulness and meditation can also promote better sleep.'

In [32]:
import time
import copy

rerank_dataset = copy.deepcopy(dataset)

for test_row in rerank_dataset:
  response = adjusted_graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(5) # To try to avoid rate limiting.

In [33]:
rerank_dataset.samples[0].eval_sample.response

"To do partial crunches to help with your back pain, follow these steps:\n\n1. Lie on your back with your knees bent.\n2. Cross your arms over your chest.\n3. Tighten your stomach muscles (engage your core).\n4. Gently lift your shoulders off the floor, leading with your upper back, not pulling with your neck.\n5. Hold this lifted position briefly.\n6. Slowly lower your shoulders back down to the starting position.\n\nYou should aim to do between 8 to 12 repetitions per set. It's best to start with one set and gradually increase if you feel comfortable. Be sure to listen to your body and stop if you experience any pain or discomfort. If your back pain is severe or persists, consult a healthcare professional before continuing these exercises."

In [34]:
rerank_evaluation_dataset = EvaluationDataset.from_pandas(rerank_dataset.to_pandas())

In [35]:
rerank_result = evaluate(
    dataset=rerank_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
rerank_result

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

{'context_recall': 0.6359, 'faithfulness': 0.7399, 'factual_correctness': 0.5891, 'answer_relevancy': 0.9575, 'context_entity_recall': 0.4361, 'noise_sensitivity_relevant': 0.0811}

### ❓ Question #2:

Which system performed better, on what metrics, and why?

##### Answer:

The latter system with reranked retrieval performed better on all the evaluated metrics, with answer_relevancy
 more than doubling. 
 baseline answer_relevancy is 0.44
 rerank answer_relevancy is 0.95

By retrieving more number of documents initially and re-ranking them results in more relevant smaller set of documents. Reranking surfaces more relevant chunks that might have been ranked lower by embeddings.

### ❓ Question #3:

What are the benefits and limitations of using synthetic data generation for RAG evaluation? Consider both the practical advantages and potential pitfalls.

##### Answer:

Advantages of using synthetic data generation:
- More comprehensive and diverse suite of tests based on the documents
- It's faster
- Less expensive than human generation, especially for specialized domains

Potential pitfalls:
- The selected query distribution and/or query types might not reflect the real user behavior
- May not be able to capture really unusual questions
- The quality of evaluation depends on the evaluator llm, and it can introduce some systemic bias or patterns.

### ❓ Question #4:

If you were building a production wellness assistant, which Ragas metrics would be most important to optimize for and why? Consider the healthcare/wellness domain specifically.

##### Answer:

Faithfulness would be the most important metric for a wellness assistant because, we want the system to answer only from the knowledge base provided, and not hallucinate anything.

Factual correctness is also important, because providing factualy incorrect data may cause real harm to users.

Context recall can also be helpful - ensuring that all relevant information is retrieved and not missed (like precautions, after-effects etc.)

## Activity #1: Implement a Different Reranking Strategy

In this activity, you'll experiment with different reranking parameters or strategies to see how they affect the evaluation metrics.

**Requirements:**
1. Modify the `retrieve_adjusted` function to use different parameters (e.g., change `k` values, try different top_n for reranking)
2. Or implement a different retrieval enhancement strategy (e.g., hybrid search, query expansion)
3. Run the evaluation and compare results with the baseline and reranking results above
4. Document your findings in the markdown cell below

In [36]:
### YOUR CODE HERE ###

# Implement your custom retrieval strategy here
# Example: modify retrieve_adjusted with different parameters

def retrieve_custom(state):
    # Your implementation here
    compressor = CohereRerank(model="rerank-v3.5", top_n=2)
    compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=adjusted_example_retriever, search_kwargs={"k": 8}
    )
    retrieved_docs = compression_retriever.invoke(state["question"])
    return {"context" : retrieved_docs}

adjusted_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_custom, generate])
adjusted_graph_builder.add_edge(START, "retrieve_custom")
adjusted_graph2 = adjusted_graph_builder.compile()

for test_row in rerank_dataset:
  response = adjusted_graph2.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(5) # To try to avoid rate limiting.

In [38]:
rerank_evaluation_dataset2 = EvaluationDataset.from_pandas(rerank_dataset.to_pandas())

rerank_result2 = evaluate(
    dataset=rerank_evaluation_dataset2,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
rerank_result2

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Exception in callback Task.__step()
handle: <Handle Task.__step()>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/asyncio/events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: cannot enter context: <_contextvars.Context object at 0x10ab56d40> is already entered
Exception in callback Task.__step()
handle: <Handle Task.__step()>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/asyncio/events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: cannot enter context: <_contextvars.Context object at 0x10ab56d40> is already entered
Task was destroyed but it is pending!
task: <Task pending name='Task-6033' coro=<_async_in_context.<locals>.run_in_context() done, defined at /Users/sireeshapulipati/AIE9

{'context_recall': 0.5777, 'faithfulness': 0.5824, 'factual_correctness': 0.5882, 'answer_relevancy': 0.9619, 'context_entity_recall': 0.3304, 'noise_sensitivity_relevant': 0.1149}

### Activity #1 Findings:

*Document your findings here: What strategy did you try? How did it compare to the baseline and reranking results?*

I changed the k value to 8 (the number of documents that the base retriever retrieves from vector store) and added top_n argument to CohereRank. The default value is 3. I'm tried with 2, to see if just returning top 2 documents instead of the default 3 would improve the performance.

Comparing the metrics:

baseline: {'context_recall': 0.2887, 'faithfulness': 0.5581, 'factual_correctness': 0.4082, 'answer_relevancy': 0.4438, 'context_entity_recall': 0.2645, 'noise_sensitivity_relevant': 0.0000}

rerank: {'context_recall': 0.6359, 'faithfulness': 0.7399, 'factual_correctness': 0.5891, 'answer_relevancy': 0.9575, 'context_entity_recall': 0.4361, 'noise_sensitivity_relevant': 0.0811}

custom rerank: {'context_recall': 0.5777, 'faithfulness': 0.5824, 'factual_correctness': 0.5882, 'answer_relevancy': 0.9619, 'context_entity_recall': 0.3304, 'noise_sensitivity_relevant': 0.1149}

The custom reranker (with k=8 and top_n = 2) provided higher relevancy, factual correctness, and faithfulness compared to baseline. However, compared to the original reranker, it is only better on answer relevance (with 0.96 vs 0.95), and performed poorly on all others.

The original rerank provided the best context recall, faithfulness, factual correctness and context entity recall.

In [40]:
#Query Expansion

from langchain.retrievers.multi_query import MultiQueryRetriever

retriever_multi_query = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),
    llm=llm,
)

def retrieve_multi_query(state):
  retrieved_docs = retriever_multi_query.invoke(state["question"])
  return {"context" : retrieved_docs}

adjusted_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_multi_query, generate])
adjusted_graph_builder.add_edge(START, "retrieve_multi_query")
adjusted_graph3 = adjusted_graph_builder.compile()

for test_row in rerank_dataset:
  response = adjusted_graph3.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(5) # To try to avoid rate limiting.

rerank_evaluation_dataset3 = EvaluationDataset.from_pandas(rerank_dataset.to_pandas())

rerank_result3 = evaluate(
    dataset=rerank_evaluation_dataset3,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
rerank_result3


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Exception raised in Job[53]: TimeoutError()


{'context_recall': 0.7229, 'faithfulness': 0.6682, 'factual_correctness': 0.5627, 'answer_relevancy': 0.8657, 'context_entity_recall': 0.4023, 'noise_sensitivity_relevant': 0.2249}

I implemented Query expansion separately without reranking. In Query expansion, the original query is expanded into multiple related queries before retrieval.

Findings:
- Multi-query approach provided the best context recall of 0.72,  higher than the both rerank retrievers.
- It also performed well on context entity recall metric with 0.40 (not far from original reranl: 0.44; the custom rerank has the lowest context entity recall with 0.33)
- This approach has the highest noise sensitivity (0.22). Indicates that it is more affected by irrelelvant context than rerank (0.08 and 0.11)

In [41]:
# Combine custom reranking with query expansion

def retrieve_combined(state):
  # First retrieve with custom reranking
  compressed_docs = retrieve_custom(state)["context"]
  
  # Then use query expansion on the compressed docs
  expanded_docs = retrieve_multi_query(state)["context"]
  
  # Combine the results
  combined_docs = compressed_docs + expanded_docs
  
  # Return the combined docs
  return {"context": combined_docs}

adjusted_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_combined, generate])
adjusted_graph_builder.add_edge(START, "retrieve_combined")
adjusted_graph4 = adjusted_graph_builder.compile()

for test_row in rerank_dataset:
  response = adjusted_graph4.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(5) # To try to avoid rate limiting.

rerank_evaluation_dataset4 = EvaluationDataset.from_pandas(rerank_dataset.to_pandas())

rerank_result4 = evaluate(
    dataset=rerank_evaluation_dataset4,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
rerank_result3

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Exception raised in Job[53]: TimeoutError()
Exception raised in Job[65]: TimeoutError()


{'context_recall': 0.7229, 'faithfulness': 0.6682, 'factual_correctness': 0.5627, 'answer_relevancy': 0.8657, 'context_entity_recall': 0.4023, 'noise_sensitivity_relevant': 0.2249}

**Combined custom reranking and query expansion**

- Context recall: Combined and QE are best (0.72 vs 0.64 for original reranker).
- Answer quality: Original reranker leads on faithfulness, factual correctness, answer relevancy, and context entity recall.
- Noise robustness: Original reranker (0.08) is much better than QE/combined (0.22).
- Combined vs QE: Metrics are identical, so the custom reranker does not appear to change the outcome yet.

